# 2. ノイズ除去

セグメンテーション結果から根以外のノイズ部分を可能な限り除去

In [1]:
import cv2
import os
import re
import numpy as np

In [33]:
SRC = '../data/cropped_segs_transformed'
DST1 = '../data/noise_removed'
DST2 = '../data/pile_up'
MAX_CNT = '../data/max_cnt'

In [34]:
def find_max_cnt(contours):
    """面積が最大の部分を取る"""
    max_idx = -1
    max_area = 0
    for i, cnt in enumerate(contours):
        if cv2.contourArea(cnt) > max_area:
            max_idx = i
            max_area = cv2.contourArea(cnt)
    cnt = contours[max_idx]
    return cnt

In [35]:
prev = ''
fnames = sorted(os.listdir(SRC))
for fname in fnames:
    img = cv2.imread(os.path.join(SRC, fname))
    #fnameのアンダーバーまでを取得
    root_name = re.findall('(.+)_', fname)[0]
    if root_name != prev:
        if fname != fnames[0]:
            print(base_img.shape)
            cv2.imwrite(os.path.join(DST2, prev +'.png'), base_img)
            cv2.imwrite(os.path.join(DST2, fname), img)
        base_img = img
    else:
        base_img = cv2.bitwise_or(base_img, img)
    
    cv2.imwrite(os.path.join(DST2, fname), base_img)
    prev = root_name
cv2.imwrite(os.path.join(DST2, root_name+'.png'), base_img)

(2600, 2600, 3)
(2600, 2600, 3)
(2600, 2600, 3)
(2600, 2600, 3)
(2600, 2600, 3)
(2600, 2600, 3)
(2600, 2600, 3)
(2600, 2600, 3)


True

In [26]:
fnames2 = sorted(os.listdir(DST2))
for fname in fnames2:
    img = cv2.imread(os.path.join(DST2, fname))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,1,255,0)
    image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnt = find_max_cnt(contours)
    blk = np.zeros(img.shape, np.uint8)
    root = cv2.drawContours(blk, [cnt], 0, (255,255,255), -1)
    cv2.imwrite(os.path.join(MAX_CNT, fname), root)

FileNotFoundError: [Errno 2] No such file or directory: '../data/pile-up'

In [36]:
for fname in fnames:
    img = cv2.imread(os.path.join(SRC, fname))
    root_name = re.findall('(.+)_', fname)[0]
    mask = cv2.imread(os.path.join(MAX_CNT, root_name + '.png'))
    img = cv2.bitwise_and(img, mask)
    cv2.imwrite(os.path.join(DST1, fname), img)

[ WARN:0@729.596] global loadsave.cpp:248 findDecoder imread_('../data/max_cnt/P_En1_05d_IMG.png'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /io/opencv/modules/core/src/arithm.cpp:212: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'binary_op'
